# Adaboost classifier

In [47]:
# Import guys
import pandas as pd
import numpy as np
from scipy import sparse
# from AdaBoostClassifier import AdaBoostClassifier
%run AdaBoostClassifier.py
from sklearn.model_selection import train_test_split



### Import data
We should leverage the sparse matrix format.

https://towardsdatascience.com/working-with-sparse-data-sets-in-pandas-and-sklearn-d26c1cfbe067

This shit is incredible

In [18]:
# Data
dat = pd.read_csv('data/cleanData.csv')
labels = dat["Category"]
dat.drop(['Category', 'cleaned_msg', 'nondupe'], inplace=True, axis = 1)
columns = dat.columns
dat = dat.to_numpy()
print(labels)

320
0      0
1      0
2      1
3      0
4      0
      ..
315    0
316    0
317    0
318    0
319    1
Name: Category, Length: 320, dtype: int64


New method

In [39]:
# column names
with open('data/column_names.txt', 'r') as f:
    column_names = [line.strip() for line in f]

sparse_dat = sparse.load_npz("data/sparse_df.npz")

In [40]:
# Extract labels from the first column
labels = sparse_dat[:, 0]

# Create a list of column indices to keep
to_keep = list(set(range(sparse_dat.shape[1])) - set([0]))

# Extract the design matrix
X = sparse_dat[:, to_keep]

print(f'labels: \n{labels}')

labels: 
  (2, 0)	1
  (5, 0)	1
  (8, 0)	1
  (9, 0)	1
  (11, 0)	1
  (12, 0)	1
  (15, 0)	1
  (19, 0)	1
  (34, 0)	1
  (42, 0)	1
  (54, 0)	1
  (56, 0)	1
  (65, 0)	1
  (67, 0)	1
  (68, 0)	1
  (93, 0)	1
  (95, 0)	1
  (114, 0)	1
  (117, 0)	1
  (120, 0)	1
  (121, 0)	1
  (123, 0)	1
  (134, 0)	1
  (135, 0)	1
  (139, 0)	1
  :	:
  (15952, 0)	1
  (15953, 0)	1
  (15954, 0)	1
  (15955, 0)	1
  (15956, 0)	1
  (15957, 0)	1
  (15958, 0)	1
  (15959, 0)	1
  (15960, 0)	1
  (15961, 0)	1
  (15962, 0)	1
  (15963, 0)	1
  (15964, 0)	1
  (15965, 0)	1
  (15966, 0)	1
  (15967, 0)	1
  (15968, 0)	1
  (15969, 0)	1
  (15970, 0)	1
  (15971, 0)	1
  (15972, 0)	1
  (15973, 0)	1
  (15974, 0)	1
  (15975, 0)	1
  (15976, 0)	1


In [41]:
print(f'Design matrix: \n{X}')

Design matrix: 
  (0, 0)	1060
  (1, 0)	3125
  (2, 0)	981
  (3, 0)	16024
  (4, 0)	2757
  (5, 0)	992
  (6, 0)	889
  (7, 0)	340
  (8, 0)	16178
  (9, 0)	1218
  (10, 0)	2151
  (11, 0)	3414
  (12, 0)	16079
  (13, 0)	2330
  (14, 0)	1680
  (15, 0)	16520
  (16, 0)	3029
  (17, 0)	866
  (18, 0)	945
  (19, 0)	873
  (20, 0)	2243
  (21, 0)	1800
  (22, 0)	3592
  (23, 0)	123
  (24, 0)	936
  :	:
  (8386, 56201)	2
  (8669, 56202)	1
  (10316, 56202)	1
  (8805, 56203)	1
  (9218, 56203)	1
  (9565, 56204)	1
  (9814, 56204)	1
  (6232, 56205)	1
  (9353, 56205)	1
  (5661, 56206)	9
  (6860, 56206)	1
  (8698, 56206)	1
  (125, 56207)	1
  (1228, 56207)	1
  (4422, 56207)	1
  (5648, 56208)	1
  (7464, 56208)	1
  (5648, 56209)	2
  (6025, 56210)	2
  (6025, 56211)	2
  (6151, 56212)	2
  (8669, 56213)	1
  (10316, 56213)	1
  (6340, 56214)	1
  (8696, 56214)	1


# Issue: 
Why the hell are these numbers so large. Something something is up with the compressed numbers thing.
Edit: Nick is a God. Something was up with the dataframe copy. He fixed it. Jonathan likes men.

### Train/test

In [42]:
# To do - stratify the split 
n_samples = labels.shape[0]
# Use train_test_split.
X_train, X_test, y_train, y_test = train_test_split(sparse_dat, labels, test_size=0.2, shuffle=False)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.dtype)
print("proportion of spam in training data:", (y_train == 1).sum().item() / y_train.shape[0])
print("proportion of spam in testing data:", (y_test == 1).sum().item() / y_test.shape[0])

(14524, 56216)
(14524, 1)
(3632, 56216)
(3632, 1)
int64
proportion of spam in training data: 0.39568989259157256
proportion of spam in testing data: 0.11921806167400881


# Model Training + Test Set Evaluation

In [50]:
def errors(y, y_pred):
        """
        Calculate the proportion of type 2 errors - when the true label is 1 - spam, and the predicted label is 0 - ham

        Args:
        y: true labels
        y_pred: predicted labels
        """
        n = y.shape[0]
        type2errors = ((y == 1) & (y_pred == 0)).sum().item()
        type1errors = ((y == 0) & (y_pred == 1)).sum().item()
        correct = (y_pred == y).sum().item()
        return type2errors, type1errors, correct

In [51]:
aboost1 = AdaBoostClassifier(n_estimators = 50, lr = 0.01, type2penalty = False, max_DT_depth = None)
aboost1.fit(X = X_train, y = y_train.toarray().ravel())

predictions = aboost1.predict(X_test)
type2, type1, correct = errors(y_test.toarray().ravel(), predictions)
print("Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')
print(aboost1.weights)

Accuracy:  1.0
unique predictions - should be 0 and 1: [0. 1.]
type 2 errors: 0 
 type 1 errors: 0
[6.8851556e-05 6.8851556e-05 6.8851556e-05 ... 6.8851556e-05 6.8851556e-05
 6.8851556e-05]


In [52]:
aboost2 = AdaBoostClassifier(n_estimators = 50, lr = 0.01, type2penalty = True, max_DT_depth = None)
aboost2.fit(X = X_train, y = y_train.toarray().ravel())

predictions = aboost2.predict(X_test)
type2, type1, correct = errors(y_test.toarray().ravel(), predictions)
print("Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')
print(aboost2.weights)

Accuracy:  1.0
unique predictions - should be 0 and 1: [0. 1.]
type 2 errors: 0 
 type 1 errors: 0
[6.8851556e-05 6.8851556e-05 6.8851556e-05 ... 6.8851556e-05 6.8851556e-05
 6.8851556e-05]


# Training Set Evaluation

In [54]:
predictions = aboost1.predict(X_train)
type2, type1, correct = errors(y_train.toarray().ravel(), predictions)
print("Model 1 (no penalty) Training set Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')

Model 1 (no penalty) Training set Accuracy:  1.0
unique predictions - should be 0 and 1: [0. 1.]
type 2 errors: 0 
 type 1 errors: 0


In [55]:
predictions = aboost2.predict(X_train)
type2, type1, correct = errors(y_train.toarray().ravel(), predictions)
print("Model 2 (penalty) Training set Accuracy: ", correct/len(predictions))
print(f'unique predictions - should be 0 and 1: {np.unique(predictions)}')
print(f'type 2 errors: {type2} \n type 1 errors: {type1}')

Model 2 (penalty) Training set Accuracy:  1.0
unique predictions - should be 0 and 1: [0. 1.]
type 2 errors: 0 
 type 1 errors: 0
